# 🎓 AI4Industry - Module 3 & 4 : RAG Vectoriel
## Mensaflow © 2025 - Formation CNAM

**Objectifs :**
- ✅ Comprendre le fonctionnement d'un RAG simple
- ✅ Tester avec les fake datas GreenPower Solutions
- ✅ Interface Gradio pour import de documents
- ✅ Chatbot conversationnel

---

## 📦 Installation des dépendances

In [ ]:
%%capture
# Installation Ollama
!sudo apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

# Installation des packages Python avec versions compatibles
!pip install -q langchain==0.3.20 langchain-community==0.3.20 langchain-core==0.3.40
!pip install -q qdrant-client==1.7.0
!pip install -q sentence-transformers
!pip install -q langchain-ollama
!pip install -q gradio
!pip install -q pypdf
!pip install -q python-docx

print("✅ Toutes les dépendances installées !")

## 🚀 Démarrage du serveur Ollama

In [ ]:
import os
import subprocess
import time

# Démarrer Ollama en arrière-plan
print("🔄 Démarrage du serveur Ollama...")
subprocess.Popen(["ollama", "serve"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
time.sleep(5)

# Télécharger Mistral 7B
print("📥 Téléchargement de Mistral 7B (peut prendre 2-3 minutes)...")
!ollama pull mistral
print("✅ Mistral prêt !")

## 🧠 Configuration du système RAG

In [ ]:
from langchain_ollama import OllamaLLM
from langchain_community.embeddings import HuggingFaceEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from langchain_community.vectorstores import Qdrant

# 1. Initialiser le LLM
print("🤖 Initialisation de Mistral...")
llm = OllamaLLM(
    model="mistral",
    base_url="http://localhost:11434",
    temperature=0.7
)

# 2. Initialiser les embeddings
print("📊 Chargement du modèle d'embeddings...")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

# 3. Initialiser Qdrant
print("🗄️ Initialisation de Qdrant...")
qdrant_client = QdrantClient(location=":memory:")
collection_name = "greenpower_docs"

# Créer la collection
qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)

# 4. Créer le vectorstore
vectorstore = Qdrant(
    client=qdrant_client,
    collection_name=collection_name,
    embeddings=embeddings
)

print("✅ Système RAG initialisé !")

## 📄 Fonctions de traitement de documents

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from PyPDF2 import PdfReader
import docx
import io

# Text splitter pour chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ". ", " ", ""]
)

def process_pdf(file):
    """Extraire texte d'un PDF"""
    try:
        pdf_reader = PdfReader(io.BytesIO(file))
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Erreur PDF: {str(e)}"

def process_docx(file):
    """Extraire texte d'un DOCX"""
    try:
        doc = docx.Document(io.BytesIO(file))
        text = "\n".join([para.text for para in doc.paragraphs])
        return text
    except Exception as e:
        return f"Erreur DOCX: {str(e)}"

def process_txt(file):
    """Extraire texte d'un TXT"""
    try:
        text = file.decode('utf-8')
        return text
    except Exception as e:
        return f"Erreur TXT: {str(e)}"

def add_documents_to_vectorstore(files, progress=None):
    """Ajouter des documents au vectorstore"""
    if not files:
        return "⚠️ Aucun fichier sélectionné"
    
    all_documents = []
    status_messages = []
    
    for i, file in enumerate(files):
        try:
            filename = file.name if hasattr(file, 'name') else f"file_{i}"
            
            # Lire le contenu du fichier
            file_content = file.read() if hasattr(file, 'read') else file
            
            # Extraire le texte selon le type
            if filename.endswith('.pdf'):
                text = process_pdf(file_content)
            elif filename.endswith('.docx'):
                text = process_docx(file_content)
            elif filename.endswith('.txt'):
                text = process_txt(file_content)
            else:
                status_messages.append(f"⚠️ {filename}: Format non supporté")
                continue
            
            if text.startswith("Erreur"):
                status_messages.append(f"❌ {filename}: {text}")
                continue
            
            # Créer un document
            doc = Document(
                page_content=text,
                metadata={"source": filename}
            )
            
            # Découper en chunks
            chunks = text_splitter.split_documents([doc])
            all_documents.extend(chunks)
            
            status_messages.append(f"✅ {filename}: {len(chunks)} chunks créés")
            
            if progress:
                progress((i + 1) / len(files))
                
        except Exception as e:
            status_messages.append(f"❌ {filename}: {str(e)}")
    
    # Ajouter tous les documents au vectorstore
    if all_documents:
        try:
            vectorstore.add_documents(all_documents)
            status_messages.append(f"\n🎉 Total: {len(all_documents)} chunks indexés dans Qdrant")
        except Exception as e:
            status_messages.append(f"\n❌ Erreur d'indexation: {str(e)}")
    
    return "\n".join(status_messages)

print("✅ Fonctions de traitement de documents prêtes")

## 💬 Fonction de Chat avec RAG

In [ ]:
def chat_with_rag(message, history):
    """Fonction de chat avec contexte RAG"""
    if not message.strip():
        return "⚠️ Veuillez entrer une question"
    
    try:
        # 1. Rechercher les documents pertinents
        relevant_docs = vectorstore.similarity_search(message, k=3)
        
        if not relevant_docs:
            return "⚠️ Aucun document n'a été indexé. Veuillez d'abord importer des documents."
        
        # 2. Construire le contexte
        context = "\n\n".join([f"Document {i+1} ({doc.metadata.get('source', 'N/A')}):\n{doc.page_content}" 
                                for i, doc in enumerate(relevant_docs)])
        
        # 3. Construire le prompt
        prompt = f"""Tu es un assistant IA spécialisé dans les produits GreenPower Solutions.

Contexte des documents:
{context}

Question de l'utilisateur: {message}

Instructions:
- Réponds uniquement en te basant sur le contexte fourni
- Si l'information n'est pas dans le contexte, dis-le clairement
- Sois précis et concis
- Cite la source si pertinent

Réponse:"""
        
        # 4. Obtenir la réponse du LLM
        response = llm.invoke(prompt)
        
        return response
        
    except Exception as e:
        return f"❌ Erreur: {str(e)}"

print("✅ Fonction de chat prête")

## 🎨 Interface Gradio

In [ ]:
import gradio as gr

# CSS personnalisé
custom_css = """
.gradio-container {
    font-family: 'Arial', sans-serif;
}
.header {
    text-align: center;
    padding: 20px;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    border-radius: 10px;
    margin-bottom: 20px;
}
"""

# Créer l'interface
with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:
    
    # Header
    gr.HTML("""
    <div class="header">
        <h1>🎓 AI4Industry - RAG Vectoriel</h1>
        <h3>Mensaflow © 2025 - Formation CNAM</h3>
        <p>Module 3 & 4 : Chatbot avec RAG + GreenPower Solutions</p>
    </div>
    """)
    
    with gr.Row():
        # Colonne gauche : Import de documents
        with gr.Column(scale=1):
            gr.Markdown("## 📂 Import de Documents")
            gr.Markdown("""
            **Formats supportés:**
            - 📄 PDF (.pdf)
            - 📝 Texte (.txt)
            - 📃 Word (.docx)
            """)
            
            file_input = gr.File(
                label="Sélectionnez vos documents",
                file_count="multiple",
                file_types=[".pdf", ".txt", ".docx"]
            )
            
            upload_btn = gr.Button("📤 Indexer les documents", variant="primary")
            upload_status = gr.Textbox(
                label="Status d'indexation",
                lines=10,
                interactive=False
            )
            
            gr.Markdown("""
            ---
            ### 📊 Statistiques
            """)
            
            stats_btn = gr.Button("🔍 Afficher statistiques")
            stats_output = gr.Textbox(
                label="Statistiques Qdrant",
                lines=5,
                interactive=False
            )
        
        # Colonne droite : Chat
        with gr.Column(scale=2):
            gr.Markdown("## 💬 Chat avec RAG")
            gr.Markdown("""
            Posez vos questions sur les documents GreenPower Solutions indexés.
            Le système utilisera le RAG pour trouver les informations pertinentes.
            """)
            
            chatbot = gr.Chatbot(
                height=500,
                label="Assistant GreenPower",
                avatar_images=(None, "🤖")
            )
            
            msg = gr.Textbox(
                label="Votre question",
                placeholder="Ex: Quels sont les produits GreenPower disponibles ?",
                lines=2
            )
            
            with gr.Row():
                submit_btn = gr.Button("🚀 Envoyer", variant="primary")
                clear_btn = gr.Button("🗑️ Effacer", variant="secondary")
            
            gr.Markdown("""
            ---
            ### 💡 Questions suggérées:
            - Quels sont les produits disponibles ?
            - Quelle est la capacité batterie du PG-M01 ?
            - Quel système convient pour un festival de 3 jours ?
            - Combien coûte le PG-P01 ?
            """)
    
    # Footer
    gr.HTML("""
    <div style="text-align: center; padding: 20px; color: #666;">
        <p><strong>Stack Technique:</strong> Mistral 7B (Ollama) + Qdrant + LangChain + Gradio</p>
        <p>Développé par Mensaflow pour AI4Industry CNAM 2025</p>
    </div>
    """)
    
    # Événements
    def get_stats():
        try:
            collection_info = qdrant_client.get_collection(collection_name)
            return f"""📊 Statistiques Qdrant:
            
📦 Collection: {collection_name}
📝 Nombre de vectors: {collection_info.vectors_count}
📏 Dimension: 384
📐 Distance: Cosine
✅ Status: Opérationnel
"""
        except Exception as e:
            return f"❌ Erreur: {str(e)}"
    
    # Lier les événements
    upload_btn.click(
        fn=add_documents_to_vectorstore,
        inputs=[file_input],
        outputs=[upload_status]
    )
    
    stats_btn.click(
        fn=get_stats,
        outputs=[stats_output]
    )
    
    msg.submit(
        fn=chat_with_rag,
        inputs=[msg, chatbot],
        outputs=[chatbot]
    ).then(
        lambda: "",
        outputs=[msg]
    )
    
    submit_btn.click(
        fn=chat_with_rag,
        inputs=[msg, chatbot],
        outputs=[chatbot]
    ).then(
        lambda: "",
        outputs=[msg]
    )
    
    clear_btn.click(lambda: None, None, chatbot, queue=False)

# Lancer l'interface
print("🚀 Lancement de l'interface Gradio...")
demo.launch(share=True, debug=True)

---
## 📚 Instructions pour les étudiants

### Étape 1 : Importer les documents GreenPower
1. Téléchargez les fake datas GreenPower depuis le dossier partagé
2. Dans la colonne de gauche, cliquez sur "Sélectionnez vos documents"
3. Sélectionnez plusieurs fichiers (PDF, TXT, DOCX)
4. Cliquez sur "📤 Indexer les documents"
5. Attendez le message de confirmation

### Étape 2 : Tester le RAG
1. Dans la zone de chat, posez une question sur GreenPower
2. Observez comment le système :
   - Recherche les documents pertinents
   - Construit le contexte
   - Génère une réponse avec Mistral

### Étape 3 : Analyser les performances
1. Cliquez sur "🔍 Afficher statistiques"
2. Notez le nombre de chunks indexés
3. Testez différentes questions et observez la qualité des réponses

### Questions à explorer :
- Le RAG trouve-t-il toujours les bonnes informations ?
- Que se passe-t-il si on pose une question hors contexte ?
- Comment améliorer la qualité du chunking ?
- Quelles sont les limites du RAG vectoriel simple ?

---
## 🎯 Objectifs pédagogiques atteints

✅ Comprendre l'architecture d'un RAG  
✅ Maîtriser le chunking de documents  
✅ Utiliser une base vectorielle (Qdrant)  
✅ Intégrer un LLM local (Ollama + Mistral)  
✅ Créer une interface utilisateur (Gradio)  

**Prochaine étape :** Module 5 - GraphRAG pour dépasser les limites du RAG vectoriel !

---
**Mensaflow © 2025 - Formation AI4Industry CNAM**